In [1]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

import datetime
import time
import pandas as pd

In [2]:
# 设置 EdgeOptions
options = EdgeOptions()
options.add_argument("--headless")  # 无头模式
# 初始化 WebDriver
driver = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install()), options=options)

today = datetime.datetime.now().strftime("%m-%d")
yesterday = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%m-%d")
companies = pd.read_csv("data/sichuan_new.csv")

In [3]:
# 访问页面
for index, row in companies.iterrows():
    # 打开 dataweb 的网址
    driver.get(row["dataweb"])
    time.sleep(2)  # 等待页面加载

    # 使用 BeautifulSoup 解析页面内容
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # 找到目标 <div id="gsgk" class="module">
    gsgk_div = soup.find("div", id="gsgk", class_="module")
    if gsgk_div:
        # 找到 <div class="tab-more"> 并提取 <a> 标签中的 href 属性
        tab_more = gsgk_div.find("div", class_="tab-more")
        if tab_more:
            link = tab_more.find("a")
            if link and "href" in link.attrs:
                companies.at[index, "executive_link"] = link["href"]  # 存储到新列
                print(link["href"])
            else:
                companies.at[index, "executive_link"] = None
        else:
            companies.at[index, "executive_link"] = None
    else:
        companies.at[index, "executive_link"] = None

# 关闭 WebDriver
driver.quit()

https://emweb.securities.eastmoney.com/pc_hsf10/pages/index.html?type=web&code=SZ002466&color=b#/gsgg
https://emweb.securities.eastmoney.com/pc_hsf10/pages/index.html?type=web&code=SZ000598&color=b#/gsgg
https://emweb.securities.eastmoney.com/pc_hsf10/pages/index.html?type=web&code=SH601838&color=b#/gsgg
https://emweb.securities.eastmoney.com/pc_hsf10/pages/index.html?type=web&code=SZ002312&color=b#/gsgg
https://emweb.securities.eastmoney.com/pc_hsf10/pages/index.html?type=web&code=SZ300841&color=b#/gsgg
https://emweb.securities.eastmoney.com/pc_hsf10/pages/index.html?type=web&code=SH600603&color=b#/gsgg
https://emweb.securities.eastmoney.com/pc_hsf10/pages/index.html?type=web&code=SZ000876&color=b#/gsgg
https://emweb.securities.eastmoney.com/pc_hsf10/pages/index.html?type=web&code=SH601208&color=b#/gsgg
https://emweb.securities.eastmoney.com/pc_hsf10/pages/index.html?type=web&code=SZ000568&color=b#/gsgg
https://emweb.securities.eastmoney.com/pc_hsf10/pages/index.html?type=web&code=SZ0

In [6]:
companies.to_csv("data/sichuan_new.csv", index=False)

In [7]:
# 设置 EdgeOptions
options = EdgeOptions()
options.add_argument("--headless")  # 无头模式
# 初始化 WebDriver
driver = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install()), options=options)

today = datetime.datetime.now().strftime("%m-%d")
yesterday = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%m-%d")
companies = pd.read_csv("data/sichuan_new.csv")

In [8]:
companies["executive_link"]

0      https://emweb.securities.eastmoney.com/pc_hsf1...
1      https://emweb.securities.eastmoney.com/pc_hsf1...
2      https://emweb.securities.eastmoney.com/pc_hsf1...
3      https://emweb.securities.eastmoney.com/pc_hsf1...
4      https://emweb.securities.eastmoney.com/pc_hsf1...
                             ...                        
169    https://emweb.securities.eastmoney.com/pc_hsf1...
170    https://emweb.securities.eastmoney.com/pc_hsf1...
171    https://emweb.securities.eastmoney.com/pc_hsf1...
172    https://emweb.securities.eastmoney.com/pc_hsf1...
173    https://emweb.securities.eastmoney.com/pc_hsf1...
Name: executive_link, Length: 174, dtype: object

In [15]:
# 示例公司 executive_link
test_link = companies["executive_link"][0]

# 初始化结果列表
executive_data = []

# 打开链接
driver.get(test_link)
time.sleep(3)  # 等待页面加载

# 使用 BeautifulSoup 解析 HTML
soup = BeautifulSoup(driver.page_source, "html.parser")

# 找到 glcjj_table
glcjj_table = soup.find("div", class_="glcjj_table")
if glcjj_table:
    rows = glcjj_table.find_all("tbody")  # 每个 tbody 是一个高管

    for row in rows:
        # 提取高管姓名
        name_tag = row.find("b", class_="tips-personalname")
        name = name_tag.text.replace("\xa0", "").strip() if name_tag else None

        # 提取个人简介
        bio_tag = row.find_all("tr")[1]
        gender = bio_tag.find_all("td")[0].text.strip() if bio_tag else None
        education = bio_tag.find_all("td")[1].text.strip() if bio_tag else None
        biography = bio_tag.find_all("td")[2].find("p").text.strip()

        # 提取年龄和职务
        age_tag = row.find_all("tr")[2].find_all("td")
        age = age_tag[0].text.strip() if age_tag else None
        position = age_tag[1].text.strip() if len(age_tag) > 1 else None

        # 提取任职时间
        tenure_tag = row.find_all("tr")[3].find("td")
        tenure = tenure_tag.text.strip() if tenure_tag else None

        # 将数据添加到结果列表
        executive_data.append({
            "name": name,
            "gender": gender,
            "education": education,
            "biography": biography,
            "age": age,
            "position": position,
            "tenure": tenure
        })

# 转换为 Pandas DataFrame
executive_df = pd.DataFrame(executive_data)

In [16]:
executive_df

,name,gender,education,biography,age,position,tenure
0,蒋安琪,性别:女,学历:硕士,"蒋安琪女士,中国国籍,生于1987年,西南财经大学工商管理硕士。蒋安琪女士自2017年2月起...",当前年龄:38,"职务:董事长,法定代表人,非独立董事",任职时间:2023-04-14至今
1,蒋卫平,性别:男,学历:本科,"蒋卫平先生:1955年出生,白族,中国国籍,大学本科学历,2011年9月获中华人民共和国机械...",当前年龄:70,"职务:董事会名誉主席,非独立董事",任职时间:2023-04-14至今
2,夏浚诚,性别:男,学历:硕士,"夏浚诚,中国籍,生于1972年,澳洲悉尼科技大学工程管理硕士,香港中文大学商学院MBA/EM...",当前年龄:53,"职务:总裁,非独立董事",任职时间:2021-01-15至今
3,邹军,性别:男,学历:硕士,"邹军,中国国籍,生于1973年,会计学硕士,中国注册会计师。邹军先生自2007年12月起担任...",当前年龄:52,"职务:执行副总裁,非独立董事,财务总监",任职时间:2007-12-20至今
4,张文宇,性别:男,学历:硕士,"张文宇,中国国籍,持有香港永居身份,生于1978年,在公司治理、企业管理、法律、投融资及财税...",当前年龄:47,"职务:副总裁,董事会秘书",任职时间:2021-12-13至今
5,李果,性别:男,学历:硕士,"李果,中国国籍,生于1983年,硕士学历。李果先生拥有8年锂行业经验,于2015年6月加入公...",当前年龄:42,职务:副总裁(副总经理),任职时间:2022-04-28至今
6,郭维,性别:男,学历:本科,"郭维,中国国籍,生于1970年,大学学历,2006年8月获遂宁市职称改革工作领导小组办公室授...",当前年龄:55,职务:执行副总裁(副总经理),任职时间:2022-04-28至今
7,熊万渝,性别:女,学历:硕士,"熊万渝,中国国籍,生于1975年,会计学学士学位,在读研究生。熊万渝女士自2021年2月起担...",当前年龄:50,职务:高级副总裁(副总经理),任职时间:2023-07-13至今
8,向川,性别:男,学历:硕士,"向川,中国国籍,生于1958年,商业经济学硕士。向川先生自2020年2月起担任公司独立董事,...",当前年龄:67,职务:独立董事,任职时间:2023-04-14至今
9,唐国琼,性别:女,学历:博士,"唐国琼,中国国籍,生于1963年,会计学博士。唐国琼女士自2020年11月起担任公司独立董事...",当前年龄:62,职务:独立董事,任职时间:2023-04-14至今


In [17]:
# 遍历每个公司的 executive_link
for _, row in companies.iterrows():
    company_name = row["name"]
    link = row["executive_link"]
    
    # 打开链接
    driver.get(link)
    time.sleep(3)  # 等待页面加载
    
    # 使用 BeautifulSoup 解析 HTML
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # 找到 glcjj_table
    glcjj_table = soup.find("div", class_="glcjj_table")
    if glcjj_table:
        rows = glcjj_table.find_all("tbody")  # 每个 tbody 是一个高管
        
        for row in rows:
            # 提取高管姓名
            name_tag = row.find("b", class_="tips-personalname")
            name = name_tag.text.replace("\xa0", "").strip() if name_tag else None

            # 提取个人简介
            bio_tag = row.find_all("tr")[1]
            gender = bio_tag.find_all("td")[0].text.strip() if bio_tag else None
            education = bio_tag.find_all("td")[1].text.strip() if bio_tag else None
            biography = bio_tag.find_all("td")[2].find("p").text.strip()

            # 提取年龄和职务
            age_tag = row.find_all("tr")[2].find_all("td")
            age = age_tag[0].text.strip() if age_tag else None
            position = age_tag[1].text.strip() if len(age_tag) > 1 else None

            # 提取任职时间
            tenure_tag = row.find_all("tr")[3].find("td")
            tenure = tenure_tag.text.strip() if tenure_tag else None

            # 将数据添加到结果列表
            executive_data.append({
                "company_name": company_name,  # 添加公司名称
                "name": name,
                "gender": gender,
                "education": education,
                "biography": biography,
                "age": age,
                "position": position,
                "tenure": tenure
            })
    print(f"{company_name}高管信息录入完成")
# 转换为 Pandas DataFrame
executive_df = pd.DataFrame(executive_data)

天齐锂业高管信息录入完成
兴蓉环境高管信息录入完成
成都银行高管信息录入完成
川发龙蟒高管信息录入完成
康华生物高管信息录入完成
广汇物流高管信息录入完成
新 希 望高管信息录入完成
东材科技高管信息录入完成
泸州老窖高管信息录入完成
川能动力高管信息录入完成
四川路桥高管信息录入完成
五 粮 液高管信息录入完成
红旗连锁高管信息录入完成
成都华微高管信息录入完成
国城矿业高管信息录入完成
雅化集团高管信息录入完成
雷电微力高管信息录入完成
宜宾纸业高管信息录入完成
海创药业...高管信息录入完成
坤恒顺维高管信息录入完成
中建环能高管信息录入完成
巨星农牧高管信息录入完成
成都先导高管信息录入完成
航发科技高管信息录入完成
云图控股高管信息录入完成
国光股份高管信息录入完成
成都燃气高管信息录入完成
德龙汇能高管信息录入完成
药易购高管信息录入完成
普瑞眼科高管信息录入完成
成飞集成高管信息录入完成
川网传媒高管信息录入完成
宏达股份高管信息录入完成
达威股份高管信息录入完成
*ST金时高管信息录入完成
迈克生物高管信息录入完成
帝欧家居高管信息录入完成
ST升达高管信息录入完成
水井坊高管信息录入完成
汇源通信高管信息录入完成
秦川物联高管信息录入完成
国光电气高管信息录入完成
纵横股份高管信息录入完成
新筑股份高管信息录入完成
安宁股份高管信息录入完成
尚纬股份高管信息录入完成
天邑股份高管信息录入完成
欧林生物高管信息录入完成
秉扬科技高管信息录入完成
中寰股份高管信息录入完成
优机股份高管信息录入完成
梓橦宫高管信息录入完成
趣睡科技高管信息录入完成
海天股份高管信息录入完成
四川双马高管信息录入完成
华融化学高管信息录入完成
华西能源高管信息录入完成
中自科技高管信息录入完成
利君股份高管信息录入完成
乐创技术高管信息录入完成
瑞奇智造高管信息录入完成
吉峰科技高管信息录入完成
中密控股高管信息录入完成
富临精工高管信息录入完成
浪莎股份高管信息录入完成
倍益康高管信息录入完成
大西洋高管信息录入完成
华塑控股高管信息录入完成
侨源股份高管信息录入完成
运达科技高管信息录入完成
港通医疗高管信息录入完成
大宏立高管信息录入完成
西菱动力高管信息录入完成
唐源电气高管信息录入完成
观想科技高管信息录入完成
四川美丰高管信息录入完成
天原股份高管信息录入完成

In [21]:
executive_df = executive_df.dropna(subset=["company_name"])

In [23]:
executive_df.to_csv("data/executive.csv", index=False)